# TD_SentimentExtractor
<img id=Analytics src="../../../../images/Sentiment.png" alt="Teradata" width="33%" align="right"  />
<p style='font-size:16px;font-family:Arial;color:#00233c'>TD_SentimentExtractor will process test to determine if text contains positive, negative or neutral sentiments.  It will score text either at the row level or sentence level. It uses a dictionary model using WordNet which is a lexical database of the English language or you can use a custom dictionary.

The function handles negated sentiments as follows:

- -1 if the sentiment is negated. For example, I am not happy.
- -1 if one word separates the sentiment and a negation word. For example, I am not very happy.
- +1 if two or more words separate the sentiment and a negation word. For example, I am not saying I am happy.
</p>

#### Connect to database and create a sample table

In [ ]:
%connect lake

In [ ]:
-- drop table demo_sentiment;    -- uncomment if executing a second time

In [ ]:
create volatile table demo_sentiment (id integer, the_comment varchar(200)) on commit preserve rows;

In [ ]:
insert into demo_sentiment values(1, 'This is a wonderful product.');
insert into demo_sentiment values(2, 'This is a horrible product. What idiot designed it?');
insert into demo_sentiment values(3, 'This product was the second one I bought');

#### Simple example with all defaults

<p style='font-size:16px;font-family:Arial;color:#00233c'>(<b>Hint:</b> you can add additional insert statements like above to try out different sentences)</p>

In [ ]:
select * from TD_SentimentExtractor (
    on demo_sentiment AS InputTable PARTITION BY ANY
    using
    TextColumn('the_comment')
    Accumulate ('ID')
) as dt order by id;

<p style='font-size:16px;font-family:Arial;color:#00233c'>Extract sentiments from a larger set of data (23k rows). Note that the default is to extract sentiments from all text in the comment_text column so there is 1 row of output for each comment_id.</p>

In [ ]:
SELECT top 10 * FROM TD_SentimentExtractor (
  ON (SELECT * FROM "DEMO_Retail"."Web_Comment") AS InputTable PARTITION BY ANY
  USING
  TextColumn ('comment_text')
  Accumulate ('comment_id', 'comment_text')
) AS dt ORDER BY comment_id;

<p style='font-size:16px;font-family:Arial;color:#00233c'>Below is a description of full options.  Note that we are setting the Analysis type as 'sentence' so there will be multiple rows of output for each row of input. The Full manual can be found by <a href="https://docs.teradata.com/r/Teradata-VantageCloud-Lake/Analyzing-Your-Data/Analytics-Database-Analytic-Functions/Text-Analytic-Functions/TD_SentimentExtractor/TD_SentimentExtractor-Syntax">clicking here</a></p> 

In [ ]:
SELECT top 20 * FROM TD_SentimentExtractor (
  ON (SELECT * FROM "DEMO_Retail"."Web_Comment") AS InputTable PARTITION BY ANY
  USING
  TextColumn ('comment_text')
  Accumulate ('comment_id', 'comment_text')
  AnalysisType ('sentence')     -- default is 'document' meaning 1 sentiment per row
  Priority ('NEGATIVE_RECALL')  -- Provide the highest priority to negative results, Other options:
                                -- 'NONE' (default), 'NEGATIVE_PRECISION', 'POSITIVE_RECALL', 'POSITIVE PRECISION'
  OutputType('ALL')             -- (default). Other options: 'POSITIVE', 'NEGATIVE'
) AS dt ORDER BY comment_id;

<footer style="padding-bottom:35px; background:#f9f9f9; border-bottom:3px solid #00233C">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2024. All Rights Reserved
        </div>
    </div>
</footer>